In [118]:
import tensorflow as tf
from tensorflow import keras
print(tf.__version__)

2.15.0


# Problem Statement

#### We want to develop a CNN model in order to categorise the images based on the 10 classes, which is the folder names

# Data Preprocessing

First, we load up the data that we have downloaded from kaggle.

In [119]:
IMG_SIZE = 400

train = tf.keras.utils.image_dataset_from_directory('/kaggle/input/nature-12k/inaturalist_12K/train', labels = 'inferred', image_size=(IMG_SIZE, IMG_SIZE), batch_size= 100)
val = tf.keras.utils.image_dataset_from_directory('/kaggle/input/nature-12k/inaturalist_12K/val', labels = 'inferred', image_size=(IMG_SIZE, IMG_SIZE), batch_size= 100)

Found 9999 files belonging to 10 classes.
Found 2000 files belonging to 10 classes.


Next, we create a variable with all of the different class names

In [120]:
classes = ['Amphibia', 'Animalia', 'Arachnida', 'Aves', 'Fungi', 'Insecta', 'Mammalia', 'Mollusca', 'Plantae', 'Reptilia']

Next, we make the data augmentation layer for our CNN. We will be making use of the v3 preprocess input for data preprocessing.

In [121]:
data_augmentation = tf.keras.Sequential([tf.keras.layers.RandomFlip("horizontal_and_vertical"),
                                                       tf.keras.layers.RandomRotation(0.25),
                                                       tf.keras.layers.RandomContrast(0.15)])

# Deep Learning Model: CNN

Now, we will start piecing together our CNN model. Firstly, we will create our callback function that we will be inputting into compile function such that we can prevent the model from training after reaching a satisfactory state, causing it to overfit.

In [122]:
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('val_accuracy') > 0.90):
      print("Cancelling training.")
      self.model.stop_training = True

callbacks = myCallback()

Next, we will be doing transfer learning with the InceptionV3 model from TensorFlow.

In [123]:
inceptionthemovie = tf.keras.applications.inception_v3.InceptionV3(
    input_shape=(IMG_SIZE, IMG_SIZE, 3),
    include_top=False
)
inceptionthemovie.trainable = False

Finally, we piece together our model and start training it.

In [124]:
input = tf.keras.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
temp = data_augmentation(input)
temp = tf.keras.applications.inception_v3.preprocess_input(temp)


temp = inceptionthemovie(temp, training=False)
temp = tf.keras.layers.GlobalAveragePooling2D()(temp)

model = tf.keras.Model(input, tf.keras.layers.Dense(10)(temp))

In [ ]:
model.compile(optimizer=tf.optimizers.Adam(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(train, validation_data=val, epochs=10, callbacks=[callbacks])

# Evaluation of Model

Firstly, the model we have now gives us a loss of the first value in the array and an accuracy of the second value in the array when the validation dataset is passed into our model.

In [ ]:
model.evaluate(val)

63/63 [==============================] - 25s 391ms/step - loss: 0.9063 - accuracy: 0.7055


[0.9063287973403931, 0.7055000066757202]

Next, we would like to plot our accuracy and loss for both the training and the validation sets on a graph based on epochs.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(5)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

TypeError: 'History' object is not subscriptable

# Hyperparameter Tuning